Replication of the paper's algorithm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from math import sqrt
import matplotlib.pyplot as plt

# 1. read the data
# u.data -> userId, movieId, rating, timestamp
# u.item -> movieId, title, releaseDate, videoReleaseDate, imdbURL, genre1..genre19 (binary)

# read ratings
rating_cols=["userId", "movieId", "rating", "timestamp"]
ratings=pd.read_csv(
    "data/u.data",
    sep="\t",
    header=None,
    names=rating_cols,
    encoding="latin-1"
)

# read movies
item_cols=[
    "movieId", "title", "releaseDate", "videoReleaseDate", "imdbURL",
    "unknown", "Action", "Adventure", "Animation", "Children",
    "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
    "FilmNoir", "Horror", "Musical", "Mystery", "Romance",
    "SciFi", "Thriller", "War", "Western"
]
movies=pd.read_csv(
    "data/u.item",
    sep="|",
    header=None,
    names=item_cols,
    encoding="latin-1"
)

# 2. split data into train/test
train_df, test_df=train_test_split(ratings, test_size=0.2, random_state=42)
train_df=train_df.reset_index(drop=True)
test_df=test_df.reset_index(drop=True)

# 3. build user x movie utility matrix from train
all_user_ids=sorted(train_df["userId"].unique())
all_movie_ids=sorted(train_df["movieId"].unique())

user_to_index={u:i for i, u in enumerate(all_user_ids)}
movie_to_index={m:i for i, m in enumerate(all_movie_ids)}

# init utility matrix
utility_matrix=np.zeros((len(all_user_ids), len(all_movie_ids)), dtype=np.float32)

# fill known ratings
for row in train_df.itertuples():
    u_idx=user_to_index[row.userId]
    m_idx=movie_to_index[row.movieId]
    utility_matrix[u_idx, m_idx]=row.rating

# 4. prepare features for k-means (19 genre columns)
# subset of movies in training set
movies_train=movies[movies["movieId"].isin(all_movie_ids)].copy()

# sort by movieId
movies_train.sort_values("movieId", inplace=True)
movies_train.reset_index(drop=True, inplace=True)

# genre columns
genre_cols=[
    "unknown", "Action", "Adventure", "Animation", "Children",
    "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
    "FilmNoir", "Horror", "Musical", "Mystery", "Romance",
    "SciFi", "Thriller", "War", "Western"
]
genre_features=movies_train[genre_cols].values # shape: (#movies_in_train, 19)

# 5. choose number of clusters via wcss (optional "elbow")
wcss=[]
K_MAX=10
for k in range(1, K_MAX+1):
    km_temp=KMeans(n_clusters=k, init='k-means++', random_state=42)
    km_temp.fit(genre_features)
    wcss.append(km_temp.inertia_)

plt.figure()
plt.plot(range(1, K_MAX+1), wcss, marker='o')
plt.title('K-Means Elbow on Movie Genres')
plt.xlabel('k (#clusters)')
plt.ylabel('WCSS')
plt.show()

# pick a cluster count

def train_and_calculate(best_k):
    # 6. cluster the movies using k-means
    kmeans=KMeans(n_clusters=best_k, init='k-means++', random_state=42)
    cluster_labels=kmeans.fit_predict(genre_features)

    # map movieId -> cluster
    movie_cluster_map=pd.DataFrame({
        "movieId":movies_train["movieId"],
        "cluster":cluster_labels
    })

    # 7. build user x cluster matrix = avg rating per user per cluster
    num_users=len(all_user_ids)
    user_cluster_matrix=np.zeros((num_users, best_k), dtype=np.float32)
    cluster_counts=np.zeros((num_users, best_k), dtype=np.float32)

    # accumulate sums of ratings
    for row in train_df.itertuples():
        u_id=row.userId
        m_id=row.movieId
        rating=row.rating

        # skip if not in training mapping
        if m_id not in movie_to_index:
            continue

        u_idx=user_to_index[u_id]

        # find movie's cluster
        cluster_id_arr=movie_cluster_map.loc[movie_cluster_map["movieId"]==m_id, "cluster"].values
        if len(cluster_id_arr)==0:
            continue
        cluster_id=cluster_id_arr[0]

        user_cluster_matrix[u_idx, cluster_id]+=rating
        cluster_counts[u_idx, cluster_id]+=1

    # convert sums to averages
    for u_idx in range(num_users):
        for c_idx in range(best_k):
            if cluster_counts[u_idx, c_idx]>0:
                user_cluster_matrix[u_idx, c_idx]/=cluster_counts[u_idx, c_idx]

    # subtract user's mean rating if they rated > 1 cluster
    user_cluster_matrix_normalized = np.copy(user_cluster_matrix)
    for u_idx in range(num_users):
        rated_cluster_count = np.sum(user_cluster_matrix[u_idx] > 0)
        if rated_cluster_count > 1:
            user_mean = np.nanmean(user_cluster_matrix[u_idx])
            for c_idx in range(best_k):
                if not np.isnan(user_cluster_matrix[u_idx, c_idx]):
                    user_cluster_matrix_normalized[u_idx, c_idx] -= user_mean


    # 8. user-user similarity via pearson correlation on user_cluster_matrix
    corr=np.corrcoef(user_cluster_matrix)
    corr=np.nan_to_num(corr) # replace NaNs with 0

    # 9. prediction (knn)
    def predict_rating(user_id, movie_id, topN=5, default_rating=3.0):
        # predict rating for (user, movie)
        if user_id not in user_to_index or movie_id not in movie_to_index:
            return default_rating

        u_idx=user_to_index[user_id]

        # find cluster for movie
        cluster_id_arr=movie_cluster_map.loc[movie_cluster_map["movieId"]==movie_id, "cluster"].values
        if len(cluster_id_arr)==0:
            return default_rating
        c_label=cluster_id_arr[0]

        # get correlation with other users
        user_sims=corr[u_idx, :]
        # sort by similarity
        neighbors_sorted=np.argsort(-user_sims)

        sim_sum=0.0
        weighted_sum=0.0
        found_neighbors=0

        for nbr_idx in neighbors_sorted:
            if nbr_idx==u_idx:
                continue

            # neighbor's rating for cluster
            nbr_rating=user_cluster_matrix[nbr_idx, c_label]

            if nbr_rating>0:
                sim=user_sims[nbr_idx]
                weighted_sum+=sim*nbr_rating
                sim_sum+=abs(sim)
                found_neighbors+=1

                if found_neighbors>=topN:
                    break

        if sim_sum==0:
            return default_rating
        return weighted_sum/sim_sum

    # 10. evaluate on test set (rmse)
    pred_ratings=[]
    true_ratings=[]

    for row in test_df.itertuples():
        user_id=row.userId
        movie_id=row.movieId
        actual=row.rating

        pred=predict_rating(user_id, movie_id, topN=5)
        pred_ratings.append(pred)
        true_ratings.append(actual)

    pred_ratings=np.array(pred_ratings)
    true_ratings=np.array(true_ratings)

    mse=np.mean((pred_ratings-true_ratings)**2)
    rmse=sqrt(mse)
    # evaluate on training set (rmse)
    train_preds = []
    train_truths = []

    for row in train_df.itertuples():
        user_id = row.userId
        movie_id = row.movieId
        actual = row.rating

        pred = predict_rating(user_id, movie_id, topN=5)
        train_preds.append(pred)
        train_truths.append(actual)

    train_preds = np.array(train_preds)
    train_truths = np.array(train_truths)
    train_mse = np.mean((train_preds - train_truths) ** 2)
    train_rmse = sqrt(train_mse)

    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Test  RMSE: {rmse:.4f}")


Below see our improved version of the paper's algorithm. We introduced a validation set to avoid the error of tuning the hyperparameters on the test set.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from math import sqrt
import matplotlib.pyplot as plt

# 1. read the data
# u.data -> userId, movieId, rating, timestamp
# u.item -> movieId, title, releaseDate, videoReleaseDate, imdbURL, genre1..genre19 (binary)

# read ratings
rating_cols=["userId", "movieId", "rating", "timestamp"]
ratings=pd.read_csv(
    "data/u.data",
    sep="\t",
    header=None,
    names=rating_cols,
    encoding="latin-1"
)

# read movies
item_cols=[
    "movieId", "title", "releaseDate", "videoReleaseDate", "imdbURL",
    "unknown", "Action", "Adventure", "Animation", "Children",
    "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
    "FilmNoir", "Horror", "Musical", "Mystery", "Romance",
    "SciFi", "Thriller", "War", "Western"
]
movies=pd.read_csv(
    "data/u.item",
    sep="|",
    header=None,
    names=item_cols,
    encoding="latin-1"
)

# 2. split data into train/validation/test
train_df, temp_df = train_test_split(ratings, test_size=0.3, random_state=42)  # 70% for training
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  # 15% for validation, 15% for test
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# 3. build user x movie utility matrix from train
all_user_ids = sorted(train_df["userId"].unique())
all_movie_ids = sorted(train_df["movieId"].unique())

user_to_index = {u: i for i, u in enumerate(all_user_ids)}
movie_to_index = {m: i for i, m in enumerate(all_movie_ids)}

# init utility matrix
utility_matrix = np.zeros((len(all_user_ids), len(all_movie_ids)), dtype=np.float32)

# fill known ratings
for row in train_df.itertuples():
    u_idx = user_to_index[row.userId]
    m_idx = movie_to_index[row.movieId]
    utility_matrix[u_idx, m_idx] = row.rating

# 4. prepare features for k-means (19 genre columns)
# subset of movies in training set
movies_train = movies[movies["movieId"].isin(all_movie_ids)].copy()

# sort by movieId
movies_train.sort_values("movieId", inplace=True)
movies_train.reset_index(drop=True, inplace=True)

# genre columns
genre_cols = [
    "unknown", "Action", "Adventure", "Animation", "Children",
    "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
    "FilmNoir", "Horror", "Musical", "Mystery", "Romance",
    "SciFi", "Thriller", "War", "Western"
]
genre_features = movies_train[genre_cols].values  # shape: (#movies_in_train, 19)

# 5. choose number of clusters via wcss (optional "elbow")
wcss = []
K_MAX = 10
for k in range(1, K_MAX + 1):
    km_temp = KMeans(n_clusters=k, init='k-means++', random_state=42)
    km_temp.fit(genre_features)
    wcss.append(km_temp.inertia_)

plt.figure()
plt.plot(range(1, K_MAX + 1), wcss, marker='o')
plt.title('K-Means Elbow on Movie Genres')
plt.xlabel('k (#clusters)')
plt.ylabel('WCSS')
plt.show()

# pick a cluster count (use validation set to decide)
def choose_best_k():
    best_rmse = float('inf')
    best_k = 2  # Default initial best k

    for k in range(2, 21):  # Try cluster counts from 2 to 20
        rmse = train_and_calculate(k, val_df)
        print(f"RMSE for k={k}: {rmse:.4f}")

        if rmse < best_rmse:
            best_rmse = rmse
            best_k = k

    print(f"Best number of clusters: {best_k} with RMSE: {best_rmse:.4f}")
    return best_k

# 6. train and calculate RMSE (based on validation data)
def train_and_calculate(best_k, df_to_use):
    # cluster the movies using k-means
    kmeans = KMeans(n_clusters=best_k, init='k-means++', random_state=42)
    cluster_labels = kmeans.fit_predict(genre_features)

    # map movieId -> cluster
    movie_cluster_map = pd.DataFrame({
        "movieId": movies_train["movieId"],
        "cluster": cluster_labels
    })

    # build user x cluster matrix = avg rating per user per cluster
    num_users = len(all_user_ids)
    user_cluster_matrix = np.zeros((num_users, best_k), dtype=np.float32)
    cluster_counts = np.zeros((num_users, best_k), dtype=np.float32)

    # accumulate sums of ratings
    for row in df_to_use.itertuples():
        u_id = row.userId
        m_id = row.movieId
        rating = row.rating

        # skip if not in training mapping
        if m_id not in movie_to_index:
            continue

        u_idx = user_to_index[u_id]

        # find movie's cluster
        cluster_id_arr = movie_cluster_map.loc[movie_cluster_map["movieId"] == m_id, "cluster"].values
        if len(cluster_id_arr) == 0:
            continue
        cluster_id = cluster_id_arr[0]

        user_cluster_matrix[u_idx, cluster_id] += rating
        cluster_counts[u_idx, cluster_id] += 1

    # convert sums to averages
    for u_idx in range(num_users):
        for c_idx in range(best_k):
            if cluster_counts[u_idx, c_idx] > 0:
                user_cluster_matrix[u_idx, c_idx] /= cluster_counts[u_idx, c_idx]

    # subtract user's mean rating if they rated > 1 cluster
    user_cluster_matrix_normalized = np.copy(user_cluster_matrix)
    for u_idx in range(num_users):
        rated_cluster_count = np.sum(user_cluster_matrix[u_idx] > 0)
        if rated_cluster_count > 1:
            user_mean = np.nanmean(user_cluster_matrix[u_idx])
            for c_idx in range(best_k):
                if not np.isnan(user_cluster_matrix[u_idx, c_idx]):
                    user_cluster_matrix_normalized[u_idx, c_idx] -= user_mean

    # user-user similarity via pearson correlation on user_cluster_matrix
    corr = np.corrcoef(user_cluster_matrix)
    corr = np.nan_to_num(corr)  # replace NaNs with 0

    # prediction (knn)
    def predict_rating(user_id, movie_id, topN=5, default_rating=3.0):
        # predict rating for (user, movie)
        if user_id not in user_to_index or movie_id not in movie_to_index:
            return default_rating

        u_idx = user_to_index[user_id]

        # find cluster for movie
        cluster_id_arr = movie_cluster_map.loc[movie_cluster_map["movieId"] == movie_id, "cluster"].values
        if len(cluster_id_arr) == 0:
            return default_rating
        c_label = cluster_id_arr[0]

        # get correlation with other users
        user_sims = corr[u_idx, :]
        # sort by similarity
        neighbors_sorted = np.argsort(-user_sims)

        sim_sum = 0.0
        weighted_sum = 0.0
        found_neighbors = 0

        for nbr_idx in neighbors_sorted:
            if nbr_idx == u_idx:
                continue

            # neighbor's rating for cluster
            nbr_rating = user_cluster_matrix[nbr_idx, c_label]

            if nbr_rating > 0:
                sim = user_sims[nbr_idx]
                weighted_sum += sim * nbr_rating
                sim_sum += abs(sim)
                found_neighbors += 1

                if found_neighbors >= topN:
                    break

        if sim_sum == 0:
            return default_rating
        return weighted_sum / sim_sum

    # evaluate on test set (rmse)
    pred_ratings = []
    true_ratings = []

    for row in test_df.itertuples():
        user_id = row.userId
        movie_id = row.movieId
        actual = row.rating

        pred = predict_rating(user_id, movie_id, topN=5)
        pred_ratings.append(pred)
        true_ratings.append(actual)

    pred_ratings = np.array(pred_ratings)
    true_ratings = np.array(true_ratings)

    mse = np.mean((pred_ratings - true_ratings) ** 2)
    rmse = sqrt(mse)
    train_preds = []
    train_truths = []

    for row in train_df.itertuples():
        user_id = row.userId
        movie_id = row.movieId
        actual = row.rating

        pred = predict_rating(user_id, movie_id, topN=5)
        train_preds.append(pred)
        train_truths.append(actual)

    train_preds = np.array(train_preds)
    train_truths = np.array(train_truths)
    train_rmse = sqrt(np.mean((train_preds - train_truths) ** 2))

    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Validation/Test RMSE: {rmse:.4f}")
    return rmse

# 7. choose best k based on validation set
best_k = choose_best_k()

# 8. final model evaluation on test set with best k
print(f"\nFinal model with {best_k} clusters:")
train_and_calculate(best_k, test_df)